In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
print(words)

["'s", 'aambal', 'about', 'allen', 'any', 'anyone', 'are', 'at', 'available', 'baker', 'bakery', 'bakkery', 'be', 'best', 'bharath', 'bulk', 'buy', 'by', 'bye', 'byee', 'byeee', 'call', 'can', 'clappana', 'cloth', 'clothes', 'clothing', 'crown', 'detail', 'discount', 'do', 'eco', 'england', 'extra', 'famous', 'for', 'good', 'goodbye', 'hay', 'hello', 'help', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'it', 'know', 'kochi', 'kollam', 'krishna', 'later', 'lekshmi', 'location', 'lulu', 'maha', 'mall', 'me', 'miss', 'more', 'my', 'name', 'near', 'need', 'of', 'offer', 'order', 'peter', 'pickle', 'place', 'pothys', 'priya', 'sangam', 'sasthas', 'see', 'shop', 'shoppping', 'should', 'solly', 'store', 'sweet', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'thiruvananthapuram', 'thiruvanathapuram', 'thiruvanathpuram', 'to', 'trend', 'visit', 'what', 'whats', 'where', 'which', 'who', 'will', 'worth', 'you', 'your']


In [6]:
classes = sorted(list(set(classes)))

In [7]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [14]:
print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Hay'], 'greeting'), (['Bye'], 'bye'), (['See', 'you', 'later'], 'bye'), (['Goodbye'], 'bye'), (['byee'], 'bye'), (['byeee'], 'bye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['Can', 'you', 'help', 'me'], 'help'), (['what', 'are', 'sweet', 'shops', 'near', 'by'], 'near'), (['sweet', 'shop', 'near', 'by'], 'near'), (['I', 'need', 'sweet', 'shops', 'near', 'by'], 'near'), (['where', 'are', 'sweet', 'shops', 'near', 'me'], 'near'), (['My', 'location', 'is', 'Clappana'], 'Clappana_sweet'), (['clappana'], 'clappana'), (['Who', 'are', 'you', '?'], 'about'), (['What', 'are', 'you', '?'], 'about'), (['Who', 'you', 'are', '?'], 'about'), (['what', 'are', 'pickle', 'shops', 'near', 'by'], 'pickle'), (['pickle', 'shop', 'near', 'by'], 'pickle'), (['I', 'need', 'pickle', 'shops', 'ne

In [8]:
print(len(classes))

30


In [15]:
# create our training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
print(training[-1])

[[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [16]:
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\Abhi Manohar\AppData\Local\Temp\ipykernel_11920\1806756406.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [17]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax


model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")


Epoch 1/200


C:\Users\Abhi Manohar\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


29/29 [==============================] - 1s 1ms/step - loss: 3.4187 - accuracy: 0.0345 
Epoch 2/200
29/29 [==============================] - 0s 1ms/step - loss: 3.3200 - accuracy: 0.1034
Epoch 3/200
29/29 [==============================] - 0s 1ms/step - loss: 3.1444 - accuracy: 0.1172
Epoch 4/200
29/29 [==============================] - 0s 1ms/step - loss: 3.0486 - accuracy: 0.1172
Epoch 5/200
29/29 [==============================] - 0s 1ms/step - loss: 2.7976 - accuracy: 0.2345
Epoch 6/200
29/29 [==============================] - 0s 1ms/step - loss: 2.5079 - accuracy: 0.2621
Epoch 7/200
29/29 [==============================] - 0s 1ms/step - loss: 2.3725 - accuracy: 0.2759
Epoch 8/200
29/29 [==============================] - 0s 1ms/step - loss: 2.1293 - accuracy: 0.4207
Epoch 9/200
29/29 [==============================] - 0s 1ms/step - loss: 1.9721 - accuracy: 0.3931
Epoch 10/200
29/29 [==============================] - 0s 1ms/step - loss: 1.8293 - accuracy: 0.4138
Epoch 11/200
29/29 [

29/29 [==============================] - 0s 1ms/step - loss: 0.1254 - accuracy: 0.9517
Epoch 84/200
29/29 [==============================] - 0s 1ms/step - loss: 0.2498 - accuracy: 0.9172
Epoch 85/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1861 - accuracy: 0.9517
Epoch 86/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1433 - accuracy: 0.9586
Epoch 87/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1124 - accuracy: 0.9724
Epoch 88/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1609 - accuracy: 0.9724
Epoch 89/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1571 - accuracy: 0.9586
Epoch 90/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1133 - accuracy: 0.9586
Epoch 91/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1128 - accuracy: 0.9724
Epoch 92/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1087 - accuracy: 0.9724
Epoch 93/200


In [18]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [19]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [20]:
def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [21]:
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [22]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [23]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [25]:
chatbot_response("CAN YOU HELP ME")

1/1 [==============================] - 0s 20ms/step


'Sure, Please tell me What do you need ?'

In [4]:
# Import the library Selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

# Make browser open in background
options = webdriver.ChromeOptions()
options.add_argument('headless')

# Create the webdriver object
browser = webdriver.Chrome(
	executable_path="C:\chromedriver_win32\chromedriver.exe",
options=options)

# Obtain the Google Map URL
url = ["https://www.google.com/maps/search/Therapist+services+in+lucknow/@26.847816,80.8777511,12z/data=!3m1!4b1"]

# Initialize variables and declare it 0
i = 0

# Create a loop for obtaining data from URLs
for i in range(len(url)):

	# Open the Google Map URL
	browser.get(url[i])

	# Obtain the title of that place
	title = browser.find_element("name",
		"x3AX1-LfntMc-header-title-title")
	print(i+1, "-", title.text)
    

	# Obtain the ratings of that place
	stars = browser.find_element_by_class_name("aMPvhf-fI6EEc-KVuj8d")
	print("The stars of restaurant are:", stars.text)

	# Obtain the description of that place
	description = browser.find_element_by_class_name("uxOu9-sTGRBb-T3yXSc")
	print("Description: ", description.text)

	# Obtain the address of that place
	address = browser.find_elements_by_class_name("CsEnBe")[0]
	print("Address: ", address.text)

	# Obtain the contact number of that place
	phone = browser.find_elements_by_class_name("CsEnBe")[-2]
	print("Contact Number: ", phone.text)

	# Obtain the reviews of that place
	review = browser.find_elements_by_class_name("OXD3gb")
	print("------------------------ Reviews --------------------")
	for j in review:
		print(j.text)
	print("\n")


C:\Users\Abhi Manohar\AppData\Local\Temp\ipykernel_5436\821972526.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="x3AX1-LfntMc-header-title-title"]"}
  (Session info: headless chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00FEDCE3+50899]
	(No symbol) [0x00F7E111]
	(No symbol) [0x00E85588]
	(No symbol) [0x00EB08F9]
	(No symbol) [0x00EB0AFB]
	(No symbol) [0x00EDF902]
	(No symbol) [0x00ECB944]
	(No symbol) [0x00EDE01C]
	(No symbol) [0x00ECB6F6]
	(No symbol) [0x00EA7708]
	(No symbol) [0x00EA886D]
	GetHandleVerifier [0x01253EAE+2566302]
	GetHandleVerifier [0x012892B1+2784417]
	GetHandleVerifier [0x0128327C+2759788]
	GetHandleVerifier [0x01085740+672048]
	(No symbol) [0x00F88872]
	(No symbol) [0x00F841C8]
	(No symbol) [0x00F842AB]
	(No symbol) [0x00F771B7]
	BaseThreadInitThunk [0x76137D49+25]
	RtlInitializeExceptionChain [0x7759B74B+107]
	RtlClearBits [0x7759B6CF+191]


In [2]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [6]:
from selenium import webdriver

options = webdriver.ChromeOptions()

options.add_argument('headless')



browser = webdriver.Chrome(options=options)



url = "https://www.google.com/maps/search/Therapist+services+in+lucknow/@26.847816,80.8777511,12z/data=!3m1!4b1"

# url = "https://www.google.com/maps/place/Lucky+Dhaba/@30.653792,76.8165233,17z/data=!3m1!4b1!4m5!3m4!1s0x390feb3e3de1a031:0x862036ab85567f75!8m2!3d30.653792!4d76.818712"



browser.get(url)



# review titles / username / Person who reviews

review_titles = browser.find_elements("name","section-review-title")

print([a.text for a in review_titles])

# review text / what did they think

review_text = browser.find_elements_by_class_name("section-review-review-content")

print([a.text for a in review_text])

# get the number of stars

stars = browser.find_elements_by_class_name("section-review-stars")

first_review_stars = stars[0]

active_stars = first_review_stars.find_elements_by_class_name("section-review-star-active")

print(f"the stars the first review got was {len(active_stars)}")

[]


AttributeError: 'WebDriver' object has no attribute 'find_elements_by_class_name'

In [14]:
#Import neccesary libraries

from selenium import webdriver              # driver to control chrome browser
import pyautogui                            
from bs4 import BeautifulSoup          # to parse the html code
import threading                       # to do multi threding
import time 
import pandas as pd                   # to store data in csv file

from selenium.webdriver.common.by import By

#enter the filename

print("Enter the filename")            # filename to store data
filename=str(input())


#intiate the chrome webdriver instance
browser=webdriver.Chrome()           # chrome instance 
record=[]

e=[]
def Selenium_extractor():
    
    time.sleep(2)
    #find_elements_by_class_name("VkpGBb")
    a=browser.find_element(By.CLASS_NAME, "LoJzbe keynav-mode-off highres screen-mode")
    time.sleep(1)
    for i in range(len(a)):
        a[i].click()
        time.sleep(2)
        source=browser.page_source
                                                          #Beautiful soup for scraping the html source
        soup=BeautifulSoup(source,'html.parser')
        try:
            Name_Html=soup.findAll('div',{"class":"a5H0ec"})
            
            name=Name_Html[0].text
            if name not in e:
                e.append(name)
                print(name)
                Phone_Html=soup.findAll('span',{"class":"Io6YTe fontBodyMedium"})    
                phone=Phone_Html[0].text
                print(phone)
            
                Address_Html=soup.findAll('span',{"class":"Io6YTe fontBodyMedium"})
             
                address=Address_Html[0].text
                #print(address)
                try:
                    Website_Html=soup.findAll('div',{"class":"QqG1Sd"})
                    web=Website_Html[0].findAll('a')
            
                    website=web[0].get('href')
                except:
                    website="Not available"
                #print(website)
                record.append((name,phone,address,website))
                df=pd.DataFrame(record,columns=['Name','Phone number','Address','Website'])  # writing data to the file
                df.to_csv(filename + '.csv',index=False,encoding='utf-8')

        except:
            print("error")
            continue
            

        
    try:
        time.sleep(1)            
        next_button=browser.find_element(By.ID, "pnnext")
       # find_element_by_id("pnnext")                  # clicking the next button
     #   find_element(By.ID, ‘id’)
        next_button.click()
        browser.implicitly_wait(2)
        time.sleep(2)
        Selenium_extractor()
    except:
        print("ERROR occured at clicking net button")
        

print("Enter the link of the page")
link=input()
browser.get(str(link)) 
time.sleep(10)                                             # pausing the program for 10 secs
Selenium_extractor()

Enter the filename
abhi
Enter the link of the page
https://www.google.com/maps/place/Dentindia+dental+and+surgical+equipment/@26.8462377,80.8388886,15.74z/data=!4m6!3m5!1s0x399bff0378380e91:0xd5672e88beb369a0!8m2!3d26.8449267!4d80.8440106!16s%2Fg%2F11q45_nj85


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".LoJzbe keynav-mode-off highres screen-mode"}
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00FEDCE3+50899]
	(No symbol) [0x00F7E111]
	(No symbol) [0x00E85588]
	(No symbol) [0x00EB08F9]
	(No symbol) [0x00EB0AFB]
	(No symbol) [0x00EDF902]
	(No symbol) [0x00ECB944]
	(No symbol) [0x00EDE01C]
	(No symbol) [0x00ECB6F6]
	(No symbol) [0x00EA7708]
	(No symbol) [0x00EA886D]
	GetHandleVerifier [0x01253EAE+2566302]
	GetHandleVerifier [0x012892B1+2784417]
	GetHandleVerifier [0x0128327C+2759788]
	GetHandleVerifier [0x01085740+672048]
	(No symbol) [0x00F88872]
	(No symbol) [0x00F841C8]
	(No symbol) [0x00F842AB]
	(No symbol) [0x00F771B7]
	BaseThreadInitThunk [0x76137D49+25]
	RtlInitializeExceptionChain [0x7759B74B+107]
	RtlClearBits [0x7759B6CF+191]


In [8]:
pip install pyautogui

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wh